In [404]:
import numpy as np
import pandas as pd

In [405]:
train = pd.read_csv('/home/bogan/Загрузки/train.csv')
test = pd.read_csv('/home/bogan/Загрузки/val.csv')

In [406]:
train = train.sample(frac=1)

In [407]:
test = test.sample(frac=1)

In [408]:
train[train == 'na'] = np.nan

In [409]:
test[test == "na"] = np.nan

In [410]:
train = train.drop(['rowc', 'colc', 'ra', 'dec'], axis=1)

In [411]:
test = test.drop(['rowc', 'colc', 'ra', 'dec'], axis=1)

In [412]:
data = train.iloc[:, 1:39]

In [413]:
y = train.iloc[:, 39]

In [414]:
def norm_to_gauss(df):
    return (df - df.mean()) * 1.0 / (df.var())

In [416]:
data = data.apply(np.float64)

In [417]:
data = data.apply(norm_to_gauss)

In [418]:
data_test = test.iloc[:, 1:39]

In [419]:
data_test = data_test.apply(np.float64)

In [420]:
data_test = data_test.apply(norm_to_gauss)

In [421]:
import sklearn

In [422]:
from sklearn.impute import MICEImputer

In [423]:
scaler = MICEImputer(n_imputations=10)

In [424]:
scaler.fit(data)

MICEImputer(imputation_order='ascending', initial_strategy='mean',
      max_value=None, min_value=None, missing_values='NaN', n_burn_in=10,
      n_imputations=10, n_nearest_features=None, predictor=None,
      random_state=None, verbose=False)

In [425]:
data = scaler.transform(data)

In [426]:
data_test = scaler.transform(data_test)

In [427]:
def argmax(preds):
    return np.asarray([np.argmax(line) for line in preds])

In [428]:
from keras.utils.np_utils import to_categorical   

cl = to_categorical(y, num_classes=3)

In [429]:
from keras.layers.advanced_activations import *

In [432]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
model = Sequential()

model.add(Dense(256, init='uniform', input_dim=38, activation=PReLU()))
model.add(Dropout(0.5))
model.add(Dense(128, init='uniform', activation=PReLU()))
model.add(Dropout(0.3))
model.add(Dense(64, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(128, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(64, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(128, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(64, init='uniform', activation=PReLU()))
model.add(Dropout(0.2))
model.add(Dense(9, init='uniform', activation=PReLU()))
model.add(Dense(3, init='uniform', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
hist = model.fit(data, cl, epochs=15, batch_size=500)

predictions = argmax(model.predict(data_test))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, input_dim=38, activation=<keras.lay..., kernel_initializer="uniform")`
  """
/usr/local/lib/python3.5/dist-packages/keras/activations.py:115: UserWarning: Do not pass a layer instance (such as PReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation=<keras.lay..., kernel_initializer="uniform")`
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation=<keras.lay..., kernel_initializer="uniform")`
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11:

Epoch 1/15
30000/30000 [==============================] - 10s 334us/step - loss: 0.6090 - acc: 0.6805
Epoch 2/15
30000/30000 [==============================] - 1s 41us/step - loss: 0.5400 - acc: 0.7213
Epoch 3/15
30000/30000 [==============================] - 1s 34us/step - loss: 0.4499 - acc: 0.7690
Epoch 4/15
30000/30000 [==============================] - 1s 35us/step - loss: 0.3884 - acc: 0.8434
Epoch 5/15
30000/30000 [==============================] - 1s 34us/step - loss: 0.3586 - acc: 0.8606
Epoch 6/15
30000/30000 [==============================] - 1s 33us/step - loss: 0.3425 - acc: 0.8681
Epoch 7/15
30000/30000 [==============================] - 1s 34us/step - loss: 0.3304 - acc: 0.8715
Epoch 8/15
30000/30000 [==============================] - 1s 34us/step - loss: 0.3244 - acc: 0.8744
Epoch 9/15
30000/30000 [==============================] - 1s 33us/step - loss: 0.3191 - acc: 0.8771
Epoch 10/15
30000/30000 [==============================] - 1s 34us/step - loss: 0.3137 - acc: 0.88

In [434]:

f1_score(true, predictions, average="macro")

0.8077173308577582

In [ ]:
model.summary()

In [430]:
true = np.asarray(test.iloc[:, 39])

In [27]:

df = pd.DataFrame(data={'objid': test.iloc[:, 0], 'prediction': predictions})

In [29]:
df.to_csv('predictions.csv', encoding='utf-8', index=False)